In [2]:
import os, shutil, csv

In [3]:
img_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_cropped' # 2022160100001_cropped.nii.gz'
seg_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2' #/2022160100001/labels.nii.gz'
out_path = '/home/jaimebarranco/Desktop/score_atlas/'

num_images = 50
subfolders = sorted([f.name for f in os.scandir(seg_path) if f.is_dir()])
name_subject_original = list()
name_subject_simplified = list()

for i in range(num_images):

    # Create output dir if not exists
    out_dir = os.path.join(out_path, f'sub_{i+1:03d}')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Image and segmentation paths
    img = f'{img_path}/{subfolders[i]}_cropped.nii.gz'
    seg = f'{seg_path}/{subfolders[i]}/labels.nii.gz'

    # csv to map original name with the simplified one
    name_subject_original.append(subfolders[i])
    name_subject_simplified.append(f'sub_{i+1:03d}')

    # Copy image and segmentation to output folder
    shutil.copy2(img, out_dir + '/' + f'sub_{i+1:03d}.nii.gz')
    shutil.copy2(seg, out_dir + '/' + f'seg_{i+1:03d}.nii.gz')

with open(f'{out_path}mapping.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Subject', 'Simplified'])
    for j in range(num_images):
        writer.writerow([name_subject_original[j], name_subject_simplified[j]])